# Hosting Strands Agents with FireworksAI models in Amazon Bedrock AgentCore Runtime

This tutorial is adapted from [AWS AgentCore Sample tutorials](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main). For more details on AWS AgentCore please visit the [AgentCore docs](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/what-is-bedrock-agentcore.html)

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a Strands Agents example running Kimi K2 0905 on FireworksAI.

### Tutorial Details

| Information         | Details                                                                       |
|:--------------------|:------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                |
| Agent type          | Single                                                                        |
| Agentic Framework   | FireworksAI                                                                   |
| LLM model           | kimi-k2-instruct-0905                                                         |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and FireworksAI Model |
| Tutorial vertical   | Cross-vertical                                                                |
| Example complexity  | Easy                                                                          |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                  |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime.

In our example we will create a very simple code-generation agent which can:

1. Read files `file_read`
2. Create and write to files `file_write`
3. Write python code into those files `code_python`

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using FireworksAI models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* FireworksAI API Key --> **Make sure to add a .env file with the FIREWORKS_API_KEY variable**
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [1]:
!make install

Installing dependencies...
uv pip install -r requirements.txt
Audited 11 packages in 89ms


## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

See the <a href="strands_agents_fireworks_ai_local.py" target="_blank">strands_agents_fireworks_ai_local.py</a> file for the local version of the agent code.

#### Invoking local agent

In [2]:
%run strands_agents_fireworks_ai_local.py '{"prompt": "Write a Python function to count letters in a string and return a dictionary with the count of each letter."}'

I'll write a Python function to count letters in a string and return a dictionary with the count of each letter.
Tool #1: code_python
```python
def count_letters(text):
    """
    Count the frequency of each letter in a string.
    
    This function counts only alphabetic characters (a-z, A-Z) and ignores
    numbers, spaces, punctuation, and other non-letter characters.
    The counting is case-insensitive.
    
    Args:
        text (str): The input string to analyze
        
    Returns:
        dict: A dictionary where keys are lowercase letters and values are
              the count of each letter in the input string
              
    Examples:
        >>> count_letters("Hello World!")
        {'h': 1, 'e': 1, 'l': 3, 'o': 2, 'w': 1, 'r': 1, 'd': 1}
        
        >>> count_letters("Python 3.9")
        {'p': 1, 'y': 1, 't': 1, 'h': 1, 'o': 1, 'n': 1}
        
        >>> count_letters("123 ABC abc")
        {'a': 2, 'b': 2, 'c': 2}
    """
    letter_counts = {}
    
    # 

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`


See the <a href="strands_agents_fireworks_ai.py" target="_blank">strands_agents_fireworks_ai.py</a> file for the runtime version of the agent code.

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [3]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "strands_fireworks_ai_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agents_fireworks_ai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/strands_agents_fireworks_ai.py, bedrock_agentcore_name=strands_agents_fireworks_ai
Configuring BedrockAgentCore agent: strands_fireworks_ai_getting_started


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated .dockerignore
Generated Dockerfile: /Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/Dockerfile
Generated .dockerignore: /Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.dockerignore
Setting 'strands_fireworks_ai_getting_started' as default agent
Bedrock AgentCore configured: /Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/Dockerfile'), dockerignore_path=PosixPath('/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.dockerignore'), runtime='None', region='us-west-2', account_id='023979239971', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
launch_result = agentcore_runtime.launch(
    env_vars={
        "OPENAI_API_BASE": "https://api.fireworks.ai/inference/v1",
        "OPENAI_API_KEY": FIREWORKS_API_KEY
    }
)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_fireworks_ai_getting_started' to account 023979239971 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_fireworks_ai_getting_started


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_fireworks_ai_getting_started


✅ ECR repository available: 023979239971.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_fireworks_ai_getting_started
Getting or creating execution role for agent: strands_fireworks_ai_getting_started
Using AWS region: us-west-2, account ID: 023979239971
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-01b95285bc
✅ Reusing existing execution role: arn:aws:iam::023979239971:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-01b95285bc
✅ Execution role available: arn:aws:iam::023979239971:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-01b95285bc
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_fireworks_ai_getting_started
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-01b95285bc
Reusing existing CodeBuild execution role: arn:aws:iam::023979239971:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-01b95285bc
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_fireworks_ai_getting_starte

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [5]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_fireworks_ai_getting_started


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload. We will invoke the python developer agent with a simple leet code problem.

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [40]:
import re
def extract_code_blocks(text):
    """
    Extract code blocks from text that are wrapped in ```language or ``` markers.
    """
    pattern = r'```(?:\w+)?\n?(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)

    # wrap in ``` ```
    matches = [f"```python\n{match}\n```" for match in matches]
    return matches

In [41]:
from IPython.display import Markdown, display

LEET_CODE_PROMPT = """
Write a Python function for the following problem:

Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
"""

invoke_response = agentcore_runtime.invoke({"prompt": LEET_CODE_PROMPT})
code_block = extract_code_blocks(invoke_response['response'][0])

# Clean and display the response
display(Markdown(code_block[0]))

```python
def two_sum(nums, target):
    """
    Return indices of the two numbers in `nums` that add up to `target`.

    Args:
        nums (list[int]): Array of integers.
        target (int): Desired sum.

    Returns:
        list[int]: Two indices `[i, j]` in any order.

    Examples:
        >>> two_sum([2, 7, 11, 15], 9)
        [0, 1]
        >>> two_sum([3, 2, 4], 6)
        [1, 2]
        >>> two_sum([3, 3], 6)
        [0, 1]
    """
    seen = {}  # value -> index
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i

```

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [45]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)